In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import ViTModel

# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from VisionTransformerModel import ViTWheatModel
from dataLoaderFunc import loadSplitData, createLoader

c:\Users\pacha\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            outputs = model(rgb_batch, dsm_batch)
            loss = criterion(outputs, label_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 100 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Compute Validation Loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)
                outputs = model(rgb_batch, dsm_batch)
                loss = criterion(outputs, label_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save Best Model
        torch.save(model.state_dict(), "vit_wheat_model.pth")

# ✅ Test the model on validation set
def test_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []

    with torch.no_grad():
        for rgb_batch, dsm_batch, label_batch in test_loader:
            rgb_batch, dsm_batch = rgb_batch.to(device), dsm_batch.to(device)
            outputs = model(rgb_batch, dsm_batch)
            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(label_batch.cpu().numpy().flatten())

    return predictions, actuals


In [3]:
train_df, val_df, test_df = loadSplitData("RGB_DSM_totEarNum.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df)

Train Size: 47840, Validation Size: 5980, Test Size: 5980
Train Batches: 2990, Validation Batches: 374, Test Batches: 374


In [4]:
# ✅ Initialize Model, Loss Function, and Optimizer
# ✅ Universal device selection (works on both Mac M2 & Windows with RTX 4060)
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

model = ViTWheatModel().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)


✅ Using device: cuda


In [5]:
# ✅ Start Training
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10 | Batch 0/2990 | Loss: 101725.0469
Epoch 1/10 | Batch 100/2990 | Loss: 24245.3613
Epoch 1/10 | Batch 200/2990 | Loss: 5811.1489
Epoch 1/10 | Batch 300/2990 | Loss: 6865.1572
Epoch 1/10 | Batch 400/2990 | Loss: 7162.7280
Epoch 1/10 | Batch 500/2990 | Loss: 6818.8828
Epoch 1/10 | Batch 600/2990 | Loss: 4650.8857
Epoch 1/10 | Batch 700/2990 | Loss: 8009.5762
Epoch 1/10 | Batch 800/2990 | Loss: 14069.7695
Epoch 1/10 | Batch 900/2990 | Loss: 8990.9277
Epoch 1/10 | Batch 1000/2990 | Loss: 9682.3545
Epoch 1/10 | Batch 1100/2990 | Loss: 11566.8027
Epoch 1/10 | Batch 1200/2990 | Loss: 7959.4253
Epoch 1/10 | Batch 1300/2990 | Loss: 4316.6172
Epoch 1/10 | Batch 1400/2990 | Loss: 21406.9707
Epoch 1/10 | Batch 1500/2990 | Loss: 12259.5908
Epoch 1/10 | Batch 1600/2990 | Loss: 18633.9980
Epoch 1/10 | Batch 1700/2990 | Loss: 8475.2109
Epoch 1/10 | Batch 1800/2990 | Loss: 8076.8135
Epoch 1/10 | Batch 1900/2990 | Loss: 8333.1670
Epoch 1/10 | Batch 2000/2990 | Loss: 10083.2979
Epoch 1/10 | Bat

In [ ]:
# ✅ Load the saved model
model.load_state_dict(torch.load("vit_wheat_model.pth"))
model.eval()

# Run test
preds, actuals = test_model(model, test_loader)

# ✅ Print sample predictions
for p, a in zip(preds[:10], actuals[:10]):
    print(f"Predicted: {p:.2f}, Actual: {a:.2f}")

Predicted: 316.97, Actual: 368.00
Predicted: 551.94, Actual: 552.00
Predicted: 313.22, Actual: 454.00
Predicted: 374.20, Actual: 281.00
Predicted: 443.07, Actual: 472.00
Predicted: 362.94, Actual: 425.00
Predicted: 271.00, Actual: 295.00
Predicted: 392.60, Actual: 430.00
Predicted: 391.84, Actual: 392.00
Predicted: 145.89, Actual: 129.00


: 